In [ ]:
import numpy as np
import openmc
import openmc.deplete
from pyne import serpent
import re
import matplotlib.pyplot as plt
from openmc.data import AVOGADRO, atomic_mass

In [ ]:
# Load OpenMC depletion results
dep_omc = openmc.deplete.Results.from_hdf5('depletion_results.h5')

# Load Serpent depletion results
dep_serpent = serpent.parse_dep('msbr_endfb71.serpent_dep.m', make_mats=False)
mat_name = []
mats = {}
for key in dep_serpent.keys():
    m = re.search('MAT_(.+?)_VOLUME', key)
    if m:
        mat_name.append(m.group(1))
z_names = dep_serpent['NAMES'][0].split()[:-2]  # zzaaam codes of isotopes
nucvec = dict(zip(z_names, dep_serpent['MAT_' + mat_name[0] + '_MDENS'][:]))

res_serpent = serpent.parse_res('msbr_endfb71.serpent_res.m')

In [ ]:
X = 'Ag110_m1'
t = dep_omc.get_atoms('1', X, 'atom/cm3', time_units='d')[0]
mdens_omc = dep_omc.get_atoms('1', X, 'atom/cm3', time_units='d')[1] * atomic_mass(X) / AVOGADRO
X = 'Ag110m'
mdens_serp = nucvec[X]

plt.plot(t, mdens_omc, label=f'{X} OpenMC')
plt.plot(t, mdens_serp, label=f'{X} Serpent')
plt.ylabel('Concentration [g/cm3]')
plt.xlabel('Time [d]')
plt.legend()

# Relative error
plt.figure()
plt.plot(t, (mdens_serp - mdens_omc)/mdens_serp)
plt.ylabel('Concentration [g/cm3]')
plt.xlabel('Time [d]')
plt.title(f'Error in OpenMC {X} concentration relative to Serpent')


In [ ]:
t, keff_omc = dep_omc.get_keff(time_units='d')
keff_omc_err = keff_omc[:,1]
keff_omc = keff_omc[:,0]

plt.figure()
plt.errorbar(t, keff_omc, yerr=keff_omc_err, label='keff OpenMC')
smap = {'analog':'ANA', 'implicit':'IMP', 'collision':'COL', 'absorption':'ABS'}
for s, v in smap.items():
    keff_serp = res_serpent[f'{v}_KEFF'][:,0]
    keff_serp_err = res_serpent[f'{v}_KEFF'][:,1]
    plt.errorbar(t, keff_serp, yerr=keff_serp_err, label=f'{s} keff Serpent')
    
plt.ylabel('Keff')
plt.xlabel('Time [d]')
plt.legend()

smap = {'analog':'ANA', 'implicit':'IMP', 'collision':'COL', 'absorption':'ABS'}
for s, v in smap.items():
    plt.figure()
    plt.plot(t, keff_omc_err, label='keff error OpenMC')
    keff_serp = res_serpent[f'{v}_KEFF'][:,0]
    keff_serp_err = res_serpent[f'{v}_KEFF'][:,1]
    plt.plot(t, np.abs((keff_serp-keff_omc)/keff_serp), label=f'OpenMC keff error relative to {s} keff in Serpent')
    plt.plot(t, np.abs((keff_omc-keff_serp)/keff_omc), label=f'Serpent {s} keff error relative to keff in OpenMC')
    plt.plot(t, keff_serp_err, label=f'{s} keff error Serpent')
    plt.ylabel('Keff error')
    plt.xlabel('Time [d]')
    plt.legend()